<a href="https://colab.research.google.com/github/Rob174/Astronomy/blob/Astronomy/AI/Mod%C3%A8le000001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fonctions de base

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/TIPE'
import os
from tensorflow.python.client import device_lib
print("Utilise le",str(device_lib.list_local_devices()[0])[15:18])
print()
print()
from google.colab import files
import tensorflow as tf
from tensorflow.python import debug as tf_debug

from tensorflow.keras.layers import Layer
import matplotlib.gridspec as gridspec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,concatenate,Subtract,Multiply,Average, Concatenate,Reshape, Add, BatchNormalization, Conv2D
from tensorflow.keras.layers import Reshape,Lambda
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import models
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Convolution2D, AveragePooling2D,MaxPooling2D
import tensorflow.keras.losses
from tensorflow.keras.layers import Flatten
from tensorflow.keras import backend as K
import numpy as np
import scipy
from PIL import Image
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import cv2
import matplotlib.pyplot as plt
images = ["Galaxies_resized/"+f for f in os.listdir("Galaxies_resized/")]
noises = []

In [0]:
def next_batch(batch_size, images,tailleAttendue,formatArray):#ATTENTION : pr tenter d'améliorer l'apprentissage, on augmente la taille minimale d'image prise
    """
    formatArray : format de sortie des données de l'image ; utiliser numpy
    """
    imageEntreeTensor = []
    imageSortieTensor = []
    while len(imageEntreeTensor) < batch_size:
        try:
            np.random.shuffle(images)#choix aléatoire de l'image
            image = cv2.imread(images[0])#Ouvre en rgb l'image nettoyée
            resizedImage = cv2.resize(image,(tailleAttendue,tailleAttendue))
            imageSortieTensor.append(np.array(resizedImage,dtype=formatArray))
            imageEntreeTensor.append(np.array(resizedImage,dtype=formatArray))
        except:
            print("Error in next_batch")
    imageEntreeTensor = np.array(imageEntreeTensor,formatArray)
    return [imageEntreeTensor,imageEntreeTensor]

In [0]:
def next_batch_bruit_voile(batch_size, images,tailleAttendue,formatArray,facteursVoile,bruitParam,plageVal=[0,255]):#ATTENTION : pr tenter d'améliorer l'apprentissage, on augmente la taille minimale d'image prise
    """
    formatArray : format de sortie des données de l'image ; utiliser numpy
    facteurVoile : liste de valeur entre 0 et 1 contenant l'atténuation pour chaque couche de l'image
    bruitParam : liste avec dans l'ordre moyenne et écart type
    """
    assert plageVal[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]-bruitParam[1] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]+bruitParam[1] <= plageVal[1]
    
    imageEntreeTensor,imageSortieTensor = next_batch(batch_size,  images,tailleAttendue,formatArray)
    imageSortieTensorCopy = np.array(imageSortieTensor,dtype=np.float32)
    for image in range(imageSortieTensorCopy.shape[0]):
        for rgbIndex in range(3):
            imageSortieTensorCopy[image,:,:,rgbIndex] *= facteursVoile[rgbIndex]
    imageSortieTensorCopy = np.clip(imageSortieTensorCopy + np.random.normal(bruitParam[0],bruitParam[1],imageSortieTensorCopy.shape),plageVal[0],plageVal[1])
    return [imageEntreeTensor,np.array(imageSortieTensorCopy,dtype=formatArray)]

In [0]:
def next_batch_bruit_voile_2(batch_size, images,tailleAttendue,formatArray,facteursVoile,bruitParam,plageVal=[0,255]):#ATTENTION : pr tenter d'améliorer l'apprentissage, on augmente la taille minimale d'image prise
    """
    La versison 2 fait les  modification sélectives de couleurs après avoir ajouté le bruit
    formatArray : format de sortie des données de l'image ; utiliser numpy
    facteurVoile : liste de valeur entre 0 et 1 contenant l'atténuation pour chaque couche de l'image
    bruitParam : liste avec dans l'ordre moyenne et écart type
    """
    assert plageVal[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]-bruitParam[1] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]+bruitParam[1] <= plageVal[1]
    
    imageEntreeTensor,imageSortieTensor = next_batch(batch_size,  images,tailleAttendue,formatArray)
    imageSortieTensorCopy = np.array(imageSortieTensor,dtype=np.float32)
    for image in range(imageSortieTensorCopy.shape[0]):
        for rgbIndex in range(3):
            imageSortieTensorCopy[image,:,:,rgbIndex] += np.random.normal(bruitParam[0],bruitParam[1])
            imageSortieTensorCopy[image,:,:,rgbIndex] *= facteursVoile[rgbIndex]
    imageSortieTensorCopy /= np.max(imageSortieTensorCopy)
    imageSortieTensorCopy *= plageVal[1]
    imageSortieTensorCopy = np.clip(imageSortieTensorCopy,plageVal[0],plageVal[1])
    return [imageEntreeTensor,np.array(imageSortieTensorCopy,dtype=formatArray)]

In [0]:
def normalisation(arrayL,plageEntree=[0,255],plageSortie=[0,1]):
    assert plageEntree != plageSortie
    assert plageEntree[1]>0 and plageSortie[1] > 0
    formatArray = [array.dtype for array in arrayL]
    L = [np.array(array,dtype=np.float) for array in arrayL]
    for i in range(len(L)):
        L[i] = np.array((L[i]-plageEntree[0])/(plageEntree[1]-plageEntree[0])*(plageSortie[1]-plageSortie[0])+plageSortie[0],formatArray[i])
    return L

In [0]:
def LarrayFloatToUint(L):
    return [np.array(array,np.uint) for array in L]

In [0]:
def tauxApprentissage(epoch,ampl,tau,lim):
    taux = ampl*10**-((epoch)/tau)
    return taux if taux > lim else lim

In [0]:
def differenceAcceptee(epoch,ampl,tau,lim):
    taux = ampl*10**-((epoch)/tau)
    return taux if taux > lim else lim

In [0]:
def convertToUint(array):
    return np.array(normalisation(array,[0,1],[0,255]),dtype=np.uint8)

In [0]:
def convertToUintL(L):
    Lresult = []
    print("Entree : ",len(L))
    for i in range(len(L)):
        Lresult.append(np.array(normalisation(L[i],[0,1],[0,255]),dtype=np.uint8))
    print("Sortie : ",len(Lresult))
    return Lresult

In [0]:
index = np.arange(0,len(images))
np.random.shuffle(index)
trainingData = [images[i] for i in index[:int(0.6*len(images))]]
evalData = [images[i] for i in index[int(0.6*len(images))-1:]]

In [0]:
# source : https://stackoverflow.com/questions/46418373/how-to-resize-interpolate-a-tensor-in-keras
def interpolation(h,w,inputTensor):
    def resize_like(inputTensor,h,w):
        return tf.image.resize_nearest_neighbor(inputTensor, [h, w])

    return Lambda(resize_like, arguments={'h':h,'w':w})(inputTensor)

In [0]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects
def SELU(x):
    return 1.0507*K.elu(x,alpha=1.67326)

get_custom_objects().update({'custom_activation': Activation(SELU)})

# A mettre pour le modèle : Activation(SELU)

In [0]:
class LRN2D(Layer):#Normalisation de réponse locale
    """
    This code is adapted from pylearn2.
    License at: https://github.com/lisa-lab/pylearn2/blob/master/LICENSE.txt
    """

    def __init__(self, alpha=1e-4, k=2, beta=0.75, n=5, **kwargs):
        if n % 2 == 0:
            raise NotImplementedError("LRN2D only works with odd n. n provided: " + str(n))
        super(LRN2D, self).__init__(**kwargs)
        self.alpha = alpha
        self.k = k
        self.beta = beta
        self.n = n

    def get_output(self, train):
        X = self.get_input(train)
        b, ch, r, c = K.shape(X)
        half_n = self.n // 2
        input_sqr = K.square(X)
        extra_channels = K.zeros((b, ch + 2 * half_n, r, c))
        input_sqr = K.concatenate([extra_channels[:, :half_n, :, :],
                                   input_sqr,
                                   extra_channels[:, half_n + ch:, :, :]],
                                  axis=1)
        scale = self.k
        for i in range(self.n):
            scale += self.alpha * input_sqr[:, i:i + ch, :, :]
        scale = scale ** self.beta
        return X / scale

    def get_config(self):
        config = {"name": self.__class__.__name__,
                  "alpha": self.alpha,
                  "k": self.k,
                  "beta": self.beta,
                  "n": self.n}
        base_config = super(LRN2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [0]:
# cf https://stackoverflow.com/questions/43784921/how-to-display-custom-images-in-tensorboard-using-keras?noredirect=1#comment85726690_43784921
def make_image(tensor):
    """
    Convert an numpy representation image to Image protobuf.
    Copied from https://github.com/lanpa/tensorboard-pytorch/
    """
    from PIL import Image
    tensor = np.stack((tensor,tensor,tensor),axis=-1)
    height, width, channel = tensor.shape # numpy.ndarray
    image = Image.fromarray(tensor)
    import io
    output = io.BytesIO()
    image.save(output, format='PNG')
    image_string = output.getvalue()
    output.close()
    CHANNEL = 1
    var = tf.Summary.Image(height=height,
                         width=width,
                         colorspace=CHANNEL,
                         encoded_image_string=image_string)
    print("var : ",var)
    return var

In [0]:
def print_shape(x):
    print("Shape : ",x.get_shape().as_list())

In [0]:
index_couches = 0

In [0]:
from graphviz import render
from graphviz import Digraph,Graph
def g_get_current_id():
	global index_couches
	return str(index_couches)
def g_get_past_id(back=1):
	global index_couches
	return str(index_couches-back)
def g_new_id():
	global index_couches
	index_couches += 1
	return str(index_couches)
def g_link(graph,id1,id2):
	graph.edge(id1,id2)
def g_auto_link(graph):
	graph.edge(g_get_past_id(),g_get_current_id())

def g_conv(graph,noyau, filtres, strides = 1, auto_connect = True):
	label = None
	if type(noyau) == int:
		label = "{Convolution | {Noyau | %d} | {Filtres | %d} | {Strides | %d}}"%(noyau,filtres,strides)
	else:
		label = "{Convolution | {Noyau | %dx%d} | {Filtres | %d} | {Strides | %d}}"%(noyau[0],noyau[1],filtres,strides)
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white",style="filled")
	if auto_connect == True:
		g_auto_link(graph)
def g_max_p(graph,noyau, auto_connect = True):
	label = "{MaxPooling | {Noyau | %d}}"%(noyau)
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white",style="filled")
	if auto_connect == True:
		g_auto_link(graph)
def g_dense(graph,filtres, strides = 1, auto_connect = True):
	label = "{Dense | {Filtres | %d} | {Strides | %d}}"%(filtres,strides)
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white",style="filled")
	if auto_connect == True:
		g_auto_link(graph)

def g_dropout(graph,taux, auto_connect = True):
	label = "{Dropout | {Rate\n(taux désg_activation) | %.3f}}"%(taux)
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white",style="filled")
	if auto_connect == True:
		g_auto_link(graph)
def g_regLoc(graph,noyau = 20, k = 2, alpha= 10**-4,beta = 0.75, auto_connect = True):
	label = "{Regularisation\nRéponse\nLocale | {Noyau | %d} | {k | %.2f} | {alpha | %.2e} | {beta | %.2e}}"%(noyau,k,alpha,beta)
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white",style="filled")
	if auto_connect == True:
		g_auto_link(graph)
def g_activation(graph,type = "SELU", auto_connect = True):
	label = "{Activation | {Type | %s}}"%(type)
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white",style="filled")
	if auto_connect == True:
		g_auto_link(graph)
def g_batch_norm(graph, auto_connect = True):
	label = "{Normalisation\nPar\nBatch}"
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white",style="filled")
	if auto_connect == True:
		g_auto_link(graph)
def g_flat(graph, auto_connect = True):
	label = "{Flatten}"
	graph.node(g_get_current_id(),shape="record",label=label,color="black",fillcolor="white",style="filled")
	if auto_connect == True:
		g_auto_link(graph)
Lmerge = []
def g_concat(graph):
    global Lmerge
    graph.node(g_get_current_id(),"Concatenate")
    for elem in Lmerge:
        graph.edge(elem,g_get_current_id())
    Lmerge = []
def g_add(graph):
    global Lmerge
    graph.node(g_get_current_id(),"+")
    for elem in Lmerge:
        graph.edge(elem,g_get_current_id())
    Lmerge = []
def g_subtract(graph):
    global Lmerge
    graph.node(g_get_current_id(),"-")
    for elem in Lmerge:
        graph.edge(elem,g_get_current_id())
    Lmerge = []
def g_proba(graph, auto_connect = True):
	graph.node(g_get_current_id(),"Probabilite")
	if auto_connect == True:
		g_auto_link(graph)
def end_graph():
	global graph
	graph.render("Modele000001")

In [0]:
def extract_inpt(inpt):
    return inpt[0],inpt[1]
def conv(inpt,k,f,s,operation=False):
    inpt,graph = extract_inpt(inpt)
    global index_couches
    couche = None
    if type(k) == int: 
        couche = Convolution2D(filters=f,kernel_size=(k,k),activation=None,strides=(s,s),padding='SAME',name="%d_conv_k%d_f%d"%(index_couches,k,f))(inpt)
    else:#Otherwise we consider that it is a tuple
        couche = Convolution2D(filters=f,kernel_size=k,activation=None,strides=(s,s),padding='SAME',name="%d_conv_k%d_%d_f%d"%(index_couches,k[0],k[1],f))(inpt)
    if operation == False:
        g_conv(graph,noyau=k,filtres=f,strides=s)
    else:
        g_conv(graph,noyau=k,filtres=f,strides=s,auto_connect=False)
        Lmerge.append(g_get_current_id())
    index_couches += 1
    return [couche,graph]
def dropout(inpt,r,operation=False):
    inpt,graph = extract_inpt(inpt)
    global index_couches
    couche = Dropout(name='%d_dropout_r%.2f'%(index_couches,r),rate=0.25)(inpt)
    if operation == False:
        g_dropout(graph,taux=r)
    else:
        g_dropout(graph,taux=r,auto_connect=False)
        Lmerge.append(g_get_current_id())
    index_couches += 1
    return [couche,graph]
def lrn(inpt,n,k,a,b,operation=False):
    inpt,graph = extract_inpt(inpt)
    global index_couches
    couche = LRN2D(name='%d_lrn_n%d_k%d_a%.2e_b%.2f'%(index_couches,n,k,a,b),n=n,k=k,alpha=a,beta=b)(inpt)
    if operation == False:
        g_regLoc(graph)
    else:
        g_regLoc(graph,auto_connect=False)
        Lmerge.append(g_get_current_id())
    index_couches += 1
    return [couche,graph]
def activ(inpt,act_type,operation=False):
    inpt,graph = extract_inpt(inpt)
    assert act_type=="SELU", "other activation functions than SELU are currently not supported"
    global index_couches
    couche = Activation(SELU,name='%d_activation_%s'%(index_couches,act_type))(inpt)
    if operation == False:
        g_activation(graph)
    else:
        g_activation(graph,auto_connect=False)
        Lmerge.append(g_get_current_id())
    index_couches += 1
    return [couche,graph]
def b_norm(inpt,operation=False):
    inpt,graph = extract_inpt(inpt)
    global index_couches
    couche = BatchNormalization(name='%d_batchnorm'%(index_couches))(inpt)
    if operation == False:
        g_batch_norm(graph)
    else:
        g_batch_norm(graph,auto_connect=False)
        Lmerge.append(g_get_current_id())
    index_couches += 1
    return [couche,graph]
def dense(inpt,f,operation=False):
    inpt,graph = extract_inpt(inpt)
    global index_couches
    couche = Dense(f,activation=None,name='%d_dense_f%d'%(index_couches,f))(inpt)
    if operation == False:
        g_dense(graph,filtres=f)
    else:
        g_dense(graph,filtres=f,auto_connect=False)
        Lmerge.append(g_get_current_id())
    index_couches += 1
    return [couche,graph]
def extract_inpt_L(L):
    L_inpt = [L[i][0] for i in range(len(L))]
    graph = L[0][1]
    return L_inpt,graph
def concat(L_inpt,operation=False):
    L_inpt,graph = extract_inpt_L(L_inpt)
    global index_couches
    couche = Concatenate(axis=-1,name='%d_merge'%(index_couches))(L_inpt)
    g_concat(graph)
    index_couches += 1
    return [couche,graph]
def subtract(L_inpt,operation=False):
    L_inpt,graph = extract_inpt_L(L_inpt)
    global index_couches
    couche = Subtract(name='%d_subtract'%(index_couches))(L_inpt)
    g_subtract(graph)
    index_couches += 1
    return [couche,graph]
def max_p(inpt,k,operation=False):
    inpt,graph = extract_inpt(inpt)
    global index_couches
    couche = MaxPooling2D(name='%d_max_p_k%d'%(index_couches,k),pool_size=k,padding='VALID')(inpt)
    if operation == False:
        g_max_p(graph,noyau=k)
    else:
        g_max_p(graph,noyau=k,auto_connect=False)
        Lmerge.append(g_get_current_id())
    index_couches += 1
    return [couche,graph]
def add(L_inpt):
    L_inpt,graph = extract_inpt_L(L_inpt)
    global index_couches
    couche = Add(name='%d_add'%(index_couches))(L_inpt) 
    g_add(graph)
    index_couches += 1
    return [couche,graph]
def flat(inpt):
    inpt,graph = extract_inpt(inpt)
    global index_couches
    couche = Flatten(name='%d_flatten'%(index_couches))(inpt)
    g_flat(graph)
    index_couches += 1
    return [couche,graph]
def proba(inpt):
    inpt,graph = extract_inpt(inpt)
    global index_couches
    couche = Activation('sigmoid',name='%d_sigmoid_proba'%(index_couches))(inpt)
    g_proba(graph)
    index_couches += 1
    return couche

In [0]:
graph = Digraph(comment='Graph', format='png')
graph.node("1","Image")

In [0]:
def magn_angle(x):
    x,graph = x[0],x[1]
    x = Lambda(lambda x:K.cast(x,dtype=tf.complex64))(x)
    x_list_magn = []
    x_list_angle = []
    for i in range(3):
        fft = Lambda(lambda x: tf.fft2d(x[:,:,:,i]), output_shape=(None,199,199))(x)
        x_list_magn.append(Lambda(lambda fft:K.expand_dims(tf.math.abs(fft),axis=-1), output_shape=(None,199,199))(fft))
        x_list_angle.append(Lambda(lambda fft: K.expand_dims(tf.math.angle(fft),axis=-1), output_shape=(None,199,199))(fft))
    magn = Concatenate()(x_list_magn)
    angle = Concatenate()(x_list_angle)
    magn = Lambda(lambda magn: K.cast(magn,dtype=tf.float32), output_shape=(None,199,199))(magn)
    angle = Lambda(lambda angle: K.cast(angle,dtype=tf.float32), output_shape=(None,199,199))(angle)
    return [magn,graph],[angle,graph]
def to_rgb(magn,angle):
    magn,graph = magn[0],magn[1]
    angle,graph = angle[0],angle[1]
    magn = Lambda(lambda magn: K.cast(magn,dtype=tf.complex64))(magn)
    angle = Lambda(lambda angle:K.cast(angle,dtype=tf.complex64))(angle)
    x_list_canal = []
    for i in range(3):
        complx_tensor = Lambda(lambda mang_angle: mang_angle[0][:,:,:,i]*tf.exp(1j*mang_angle[1][:,:,:,i]))([magn,angle])
        ifft = Lambda(tf.ifft2d)(complx_tensor)
        x_list_canal.append(K.expand_dims(ifft,axis=-1))
    images = Concatenate()(x_list_canal)
    images = K.cast(images,dtype=tf.float32)
    return [images,graph]

In [0]:
def inception(inpt):
    coucheT0 = conv(inpt,k=1,f=100,s=1,operation=True)
    coucheT1 = conv(inpt,k=1,f=100,s=1)
    coucheT1 = conv(inpt,k=(1,3),f=100,s=1)
    coucheT1 = conv(inpt,k=(3,1),f=100,s=1,operation=True)
    couche = concat([coucheT0,coucheT1])
    couche = conv(inpt,k=1,f=100,s=1)
    couche = add([couche,inpt])
    return couche

In [0]:
nb_couches = 3
def AI_gen_disc_or_disc_without_input(inpt,part='gen_disc'):
    inpt = Lambda(lambda x:K.cast(x,dtype=tf.float32))(inpt)
    global graph
    inpt = [inpt,graph]
    if part == 'gen_disc' or part == 'gen':
        with K.name_scope('Generateur_FFT'):
            magn,angle = magn_angle(inpt)
            
            def analyse(couche):
                couche = conv(couche,k=2,f=500,s=1)
                couche = dropout(couche,r=0.25)
                couche = lrn(inpt=couche,n=21,k=2,a=10**-4,b=0.75)
                couche = activ(couche,act_type="SELU")
                couche = conv(couche,k=2,f=100,s=1)
                couche = b_norm(couche)
                couche = activ(couche,act_type="SELU")
                couche = dense(couche,f=100)
                return couche
            magn = analyse(magn)
            angle = analyse(angle)
            fft_back = to_rgb(magn,angle)
            
        with K.name_scope('Generateur_image'):
            couche = conv(inpt,k=2,f=500,s=1)
            couche = dropout(couche,r=0.25)
            couche = lrn(couche,n=21,k=2,a=10**-4,b=0.75)
            couche = activ(couche,act_type="SELU")
            couche = conv(couche,k=2,f=100,s=1)
            couche = b_norm(couche)
            couche = activ(couche,act_type="SELU")
            couche = conv(couche,k=2,f=100,s=1)
            couche = b_norm(couche)
            couche = activ(couche,act_type="SELU")
            
            couche = concat([couche,fft_back])
            couche = conv(couche,k=2,f=100,s=1)
            couche = b_norm(couche)
            couche = activ(couche,act_type="SELU")
            couche = conv(couche,k=2,f=100,s=1)
            couche = b_norm(couche)
            couche = activ(couche,act_type="SELU")
            couche = dense(couche,f=3)
            couche = b_norm(couche)
            couche = activ(couche,act_type="SELU")
            
            if part == 'gen':
                return couche[0]
    if part == 'gen_disc' or part == 'disc':
        with K.name_scope('Discriminateur'):
            if part == 'gen_disc':
                coucheAdaptation = conv(couche,k=2,f=100,s=1)
            else:
                coucheAdaptation = conv(inpt,k=2,f=100,s=1)
            image = inception(coucheAdaptation)
            image = lrn(image,n=21,k=2,a=10**-4,b=0.75)
            image = b_norm(image)
            image = activ(image,act_type="SELU")
            image = max_p(image,k=3)
            image = inception(image)
            image = lrn(image,n=21,k=2,a=10**-4,b=0.75)
            image = b_norm(image)
            image = activ(image,act_type="SELU")
            image = max_p(image,k=3)
            image = inception(image)
            image = lrn(image,n=21,k=2,a=10**-4,b=0.75)
            image = b_norm(image)
            image = activ(image,act_type="SELU")
            image = inception(image)
            image = lrn(image,n=21,k=2,a=10**-4,b=0.75)
            image = b_norm(image)
            image = activ(image,act_type="SELU")
            image = max_p(image,k=3,operation=False)
        
            resultatAnalyse = flat(image)
            resultatAnalyse = dense(resultatAnalyse,f=10)
            resultatAnalyse = dropout(resultatAnalyse,r=0.25)
            resultatAnalyse = activ(resultatAnalyse,act_type="SELU")
            resultatAnalyse = dense(resultatAnalyse,f=1)
            probabilite = proba(resultatAnalyse)
            end_graph()
        return probabilite

In [0]:
def AI():
    inpt = Input(shape = (199, 199, nb_couches),name=str(0)+'_'+'Image')
    probabilite_gen_disc = AI_gen_disc_or_disc_without_input(inpt,part='gen_disc')
    probabilite_disc = AI_gen_disc_or_disc_without_input(inpt,part='disc')
    generated_image = AI_gen_disc_or_disc_without_input(inpt,part='gen')
    return Model(inputs=inpt,outputs=probabilite_gen_disc),Model(inputs=inpt,outputs=probabilite_disc), Model(inputs=inpt,outputs=generated_image)

In [0]:
def mean_gen_disc_top_27(disc,gen,gen_disc,input_image,desired_image,indexModele):
    path_model = "Models/Modele"+str(indexModele)+"_"
    partial_path_gen_backup = path_model + 'gen' + '_'
    partial_path_gen_disc_backup = path_model + 'gen_disc' + '_'
    partial_path_disc_backup = path_model + 'disc' + '_'

    Lloss = []
    Lloss_gen = []
    Lloss_gen_disc = []
    Lloss_disc = []
    
    try:
        gen.save_weights(partial_path_gen_backup + 'orig.h5')
    except:
        gen.save_weights(partial_path_gen_backup + 'orig.h5')
    try:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    except:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    try:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')
    except:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')

    for i in range(3):
        gen_backup = partial_path_gen_backup + str(i) + '.h5'
        print("Training "+gen_backup)
        try:
            gen.load_weights(partial_path_gen_backup + 'orig.h5')
        except:
            gen.load_weights(partial_path_gen_backup + 'orig.h5')
        loss = gen.train_on_batch(input_image,desired_image)
        try:
            gen.save_weights(gen_backup)
        except:
            gen.save_weights(gen_backup)
        Lloss.append([gen_backup,loss])
    try:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    except:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    try:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')
    except:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')
    print("Lloss : ",Lloss)
    desired_probability = [1]*input_image.shape[0] # Batch size
    Lloss_gen_disc_tmp = []
    for index_gen_disc,path_loss_backup in enumerate(Lloss):
        try:
            gen.load_weights(path_loss_backup[0])
        except:
            gen.load_weights(path_loss_backup[0])
        for i in range(3):
            gen_disc_backup = partial_path_gen_disc_backup + str(i) + '.h5'
            print("Training "+gen_disc_backup)
            try:
                gen_disc.load_weights(partial_path_gen_disc_backup + 'orig.h5')
            except:
                gen_disc.load_weights(partial_path_gen_disc_backup + 'orig.h5')
            loss = gen_disc.train_on_batch(input_image,desired_probability)
            try:
                gen_disc.save_weights(gen_disc_backup)
            except:
                gen_disc.save_weights(gen_disc_backup)
            Lloss_gen_disc_tmp.append(path_loss_backup + [gen_disc_backup,loss])
    Lloss = Lloss_gen_disc_tmp
    print("Lloss : ",Lloss)

    desired_probability = [1]*(input_image.shape[0]//2)+[0]*(input_image.shape[0]//2)
    Lloss_disc_tmp = []
    for index_model,item in enumerate(Lloss):
        for i in range(3):
            disc_backup = partial_path_disc_backup + str(i) + '.h5'
            print("Training "+disc_backup)
            try:
                gen_disc.load_weights(Lloss[index_model][-2])
            except:
                gen_disc.load_weights(Lloss[index_model][-2])
            loss = disc.train_on_batch(input_image,desired_probability)
            try:
                disc.save_weights(disc_backup)
            except:
                disc.save_weights(disc_backup)
            Lloss_disc_tmp.append(Lloss[index_model]+[disc_backup,loss])


    Lloss = [Lloss_disc_tmp[i] + [(Lloss_disc_tmp[i][1]+Lloss_disc_tmp[i][3]+Lloss_disc_tmp[i][5])/3] for i in range(len(Lloss_disc_tmp))]
    print("Lloss : ",Lloss)
    Lloss.sort(key=lambda elem:elem[-1])
    print("Lloss sorted : ",Lloss)
    try:
        gen_disc.load_weights(Lloss[0][2])
        disc.load_weights(Lloss[0][-3])
    except:
        gen_disc.load_weights(Lloss[0][2])
        disc.load_weights(Lloss[0][-3])
    print("Best model back")
    return Lloss[0]

In [0]:
def sauvegardeModele(entree_pure,entree_deterioree,model,iteration_entrainement,summary_writer,batch_size=7):
    global Llayers
    for canal_image in range(3):
        summary_image = tf.Summary(value=[tf.Summary.Value(tag="input_pure_canal_"+str(canal_image), 
                                image=make_image(np.array(entree_pure[0,:,:,canal_image]*255,dtype=np.uint8)))])
        summary_writer.add_summary(summary_image,iteration_entrainement)
        summary_image = tf.Summary(value=[tf.Summary.Value(tag="input_deterioree_canal_"+str(canal_image), 
                                image=make_image(np.array(entree_deterioree[0,:,:,canal_image]*255,dtype=np.uint8)))])
        summary_writer.add_summary(summary_image,iteration_entrainement)
        
    for p,entree in enumerate([entree_pure,entree_deterioree]):
        layer_outputs,layer_names = [Llayers[i][0] for i in range(len(Llayers))],[Llayers[i][1] for i in range(len(Llayers))]
        model_calcul_image = Model(inputs=model.input,outputs=[model.output]+layer_outputs)
        sorties_couches = model_calcul_image.predict(entree, batch_size=batch_size)[1:] if len(layer_outputs) > 0 else [model_calcul_image.predict(entree, batch_size=batch_size)][1:]
        
        for index_couche,sortie_couche in enumerate(sorties_couches):
            layer_name = layer_names[index_couche]
            dim_sortie = sortie_couche.shape
            if len(dim_sortie) == 4:
                for canal_image in range(dim_sortie[-1]):
                    tag = layer_name
                    tag += 'pure' if p == 0 else 'deterioree'
                    tag += "_canal_" + str(canal_image)
                    summary_image = tf.Summary(value=[tf.Summary.Value(tag=tag, 
                                            image=make_image(np.array(sortie_couche[0,:,:,canal_image]*255,dtype=np.uint8)))])
                    summary_writer.add_summary(summary_image,iteration_entrainement)
    return None

In [0]:
from google.colab import output
def beep():
    output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
    return None

In [0]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time
continuer = None
def block_or_continue():
    beep()
    print("Would you resume training ? True or False ?")
    return bool(input())

In [0]:
def backup(image,imageBruitee,gen_disc,disc,index,summary_writer):
    x = np.concatenate((image,imageBruitee))
    sauvegardeModele(image,imageBruitee,gen_disc,index,summary_writer)
    p_gen_disc = gen_disc.predict(x)[0]
    p_disc = disc.predict(x)[0]

    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité gen-disc img 0 (orig pure) pure : ", 
                                        simple_value=p_gen_disc) ])
    summary_writer.add_summary(summary_loss,index)
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité disc img 0 (orig pure) pure : ", 
                                        simple_value=p_disc) ])
    summary_writer.add_summary(summary_loss,index)
    x_proba = np.concatenate((imageBruitee,imageBruitee))
    p_gen_disc = gen_disc.predict(x_proba)[0]
    p_disc = disc.predict(x_proba)[0]
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité gen-disc img 0 (orig détériorée) pure : ", 
                                        simple_value=p_gen_disc) ])
    summary_writer.add_summary(summary_loss,index)
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité disc img 0 (orig détériorée) pure : ", 
                                        simple_value=p_disc) ])
    summary_writer.add_summary(summary_loss,index)

In [0]:
def custom_loss(y_true,y_pred):
    return K.mean(K.square(y_pred - y_true),axis=-1)+K.max(K.square(y_pred - y_true),axis=-1)

In [0]:
def to_fft(img_tensor):
    L = []
    print(img_tensor.shape[0],img_tensor.get_shape().as_list())
    for index_image in range(img_tensor.get_shape().as_list()[0]):
        L.append(to_fft_single(img_tensor[index_image,:,:,:]))
    return np.stack(L,axis=0)

def to_img(fft_cplx):
    return to_uint8_array(np.fft.ifft2(fft_cplx))
def to_fft_single(img):
    fft_cplx = np.fft.fft2(img)
    fft = 20*np.log(np.abs(fft_cplx)+10**-5*np.ones(img.shape))
    fft = fft + np.abs(np.min(fft))*np.ones(fft.shape)
    fft = fft / (np.max(fft)*np.ones(fft.shape))*(255*np.ones(fft.shape))
    fft_uint8 = to_uint8_array(fft)
    print(np.max(fft_uint8),np.min(fft_uint8),fft_uint8.shape,fft_uint8.dtype)
    return fft_cplx

def to_uint8_array(array):
    return array.astype(np.uint8)
def next_batch_bruit_voile_fft():
    image,imageBruitee = next_batch_bruit_voile_2(7,images,199,np.float32,[1,1,1],[50,50])
    

In [0]:
index_couches = 0
Llayers = []
def train_gen_disc(images,eval_data,indexModele):
    gen_disc,disc,gen = AI()
    gen_disc_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    disc_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    gen_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    disc.compile(loss='binary_crossentropy', optimizer=disc_optim)
    gen_disc.compile(loss='binary_crossentropy', optimizer=gen_disc_optim)
    gen.compile(loss=custom_loss, optimizer=gen_optim)
    print("Layers : ",Llayers)
    print(gen_disc.summary())
    print(disc.summary())
    print(Llayers)
    Lloss = []
    iterations = 200
    index = 0
    import datetime
    chaine_date = datetime.datetime.today().strftime('%Y-%m-%d_%Hh%Mmin%Ss') #cf http://strftime.org/ et https://www.science-emergence.com/Articles/Obtenir-la-date-daujourdhui-au-format-YYYY-MM-DD-avec-python/
    summary_writer = tf.summary.FileWriter(logdir='./logs/Modele'+str(indexModele)+'_gen_disc_'+chaine_date,graph=tf.get_default_graph())
    while index < iterations:
        image,imageBruitee = next_batch_bruit_voile_2(7,images,199,np.float32,[1,1,1],[50,50])
        # image,imageBruitee = to_fft_single(image),to_fft_single(imageBruitee)
        current_learning_rate = tauxApprentissage(index,10**-4,10,10**-7)

        K.set_value(disc.optimizer.lr, current_learning_rate)
        K.set_value(gen_disc.optimizer.lr, current_learning_rate)
        current_learning_rate = tauxApprentissage(index,2,25,10**-7)
        K.set_value(gen.optimizer.lr, current_learning_rate)

        x = np.concatenate((image,imageBruitee))
        loss = mean_gen_disc_top_27(disc,gen,gen_disc,np.concatenate((image,imageBruitee)),np.concatenate((image,image)),indexModele)
        Lloss.append(loss)
        print("Iteration %i : gen, erreur : %f, gen_disc, erreur : %f ; disc, erreur : %f ; erreur moyenne : %f"%(index,loss[1],loss[3],loss[-2],loss[-1]))
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur gen : MSE", 
                                             simple_value=loss[1]) ])
        summary_writer.add_summary(summary_loss,index)
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur gen-disc : binary-crossentropy", 
                                             simple_value=loss[3]) ])
        summary_writer.add_summary(summary_loss,index)
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur disc : binary-crossentropy", 
                                             simple_value=loss[-2]) ])
        summary_writer.add_summary(summary_loss,index)
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur moyenne : binary-crossentropy", 
                                             simple_value=loss[-1]) ])
        summary_writer.add_summary(summary_loss,index)
        
        if index % 5 == 0:
            backup(image,imageBruitee,gen_disc,disc,index,summary_writer)
        index += 1
    summary_writer.close()
Lloss = train_gen_disc(images,evalData,9)
# See https://stackoverflow.com/questions/48242585/tf-gradients-wont-work-with-tf-assign-but-works-with